In [1]:
import requests
import json

from clarifai.rest import ClarifaiApp

from bs4 import BeautifulSoup
import requests
from pandas import Series,DataFrame

from collections import Counter

import pickle

import re

In [2]:

app = ClarifaiApp()
model = app.models.get('general-v1.3')

In [3]:

response = model.predict_by_url(url='https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/Tokyo_Tower_and_around_Skyscrapers.jpg/238px-Tokyo_Tower_and_around_Skyscrapers.jpg')
    

concepts = response['outputs'][0]['data']['concepts']
for concept in concepts:
    print(concept['name'], concept['value'])

    

skyscraper 0.9986655
city 0.99813396
skyline 0.99731296
cityscape 0.99724084
tallest 0.99628556
architecture 0.9935961
tower 0.99185944
downtown 0.9860486
office 0.9857589
travel 0.9854231
modern 0.9831452
panoramic 0.9794986
building 0.9794082
urban 0.97695947
business 0.973348
sky 0.9704137
no person 0.96794486
high 0.9670117
dusk 0.9601188
landmark 0.95253515


In [4]:
url = "https://en.wikipedia.org/wiki/Bergamo"








In [5]:
def count_words_wikipedia(name, words):
    startURL = 'https://en.wikipedia.org/wiki/'
    result = requests.get(startURL + name)

    c = result.content

    soup = BeautifulSoup(c, "lxml")

    city_name = soup.find(id='firstHeading').text
    text = soup.find(id='bodyContent').text

    regx = r'\b' + '|'.join(words) + r'\b'
    occ = re.findall(regx, text, re.IGNORECASE)
    occ_len = len(occ)
    article_len = len(text)
    score = float(occ_len) / float(article_len)

    return score





In [6]:
def find_best_words(image):
    response = model.predict_by_url(url=image)

    concepts = response['outputs'][0]['data']['concepts']
    return list(map(lambda c: c['name'], concepts))[:10]
    
    
def get_city_score(city, words):
    score = count_words_wikipedia(city, words)
    return score

#cities = ['Paris', 'Rome', 'Tokyo', 'London', 'Beijing', 'Seoul', 'New York', 'Cairo', 'Bergamo']

image = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/Tokyo_Tower_and_around_Skyscrapers.jpg/238px-Tokyo_Tower_and_around_Skyscrapers.jpg'
words = find_best_words(image)

In [7]:
def get_most_likely_cities(image, cities):
    words = find_best_words(image)
    cities_score = []
    for city in cities:
        cities_score.append((city, get_city_score(city, words)))
    return cities_score



In [8]:
def select_best_cities_from_image(image, citiess, number):
    words = find_best_words(image)
    print(words)
    cities_score = []
    for city in citiess:
        cities_score.append((city, get_city_score(city, words)))
    return sorted(cities_score, key=lambda tup: tup[1], reverse=True)
tokyo = image


def select_best_cities_fixed(image, number):
    cities = ['Paris', 'Rome', 'Tokyo', 'London', 'Beijing', 'Seoul', 'New York', 'Cairo', 'Bergamo', 'Rio de Janeiro']
    return select_best_cities_from_image(image, cities, number)

In [9]:
country = 'CH'
currency = 'CHF'
locale = 'en_GB'
json_data = requests.get('http://partners.api.skyscanner.net/apiservices/geo/v1.0?apiKey=ha712564909427747796218296388326')

parsed = json.loads(json_data.text)


In [10]:

cont = parsed['Continents']
print(type(cont))
count = cont[0]
print(type(count))
count2 = count['Countries']
print(type(count2))
count3 = count2[0]
print(type(count3))
count4 = count3['Cities']
count5 = count4[0]
print(count5)


<class 'list'>
<class 'dict'>
<class 'list'>
<class 'dict'>
{'SingleAirportCity': True, 'Airports': [{'CityId': 'BINA', 'CountryId': 'AF', 'Location': '67.823611, 34.804167', 'Id': 'BIN', 'Name': 'Bamiyan'}], 'CountryId': 'AF', 'Location': '67.823611, 34.804167', 'IataCode': 'BIN', 'Id': 'BINA', 'Name': 'Bamiyan'}


In [11]:
city_dict = {}
for continent in cont:
    for countries in continent['Countries']:
        country = countries['Name']
        for cities in countries['Cities']:
            city = cities['Name']
            airport_list = []
            for airport in cities['Airports']:
                airport_list.append(airport['Id'])
                #print(city + ', ' + country +':    ' + airports['Name'] +' '+ airports['Id'])
            city_dict[city] = airport_list

print(city_dict["Paris"])    

['BVA', 'CDG', 'ORY']


In [12]:
print(list(city_dict.keys()))

['Bamiyan', 'Bost', 'Chakcharan', 'Farah', 'Faizabad', 'Herat', 'Kabul', 'Kandahar', 'Maimana', 'Mazar-I-Sharif', 'Sharana', 'Kunduz', 'Zaranj', 'Yerevan', 'Gyoumri', 'Baku', 'Gyandzha', 'Lankaran', 'Nakhichevan', 'Qabala', 'Barisal', 'Chittagong', "Cox's Bazar", 'Dhaka', 'Jessore', 'Rajshahi', 'Saidpur', 'Sylhet', 'Jakar', 'Gelephu', 'Paro', 'Baise', 'Aba/Hongyuan', 'Arxan', 'Qionghai', 'Bijie', 'Beijing', 'Bole', 'Qinhuangdao', 'Ba Tong', 'Burqin', 'Altay', 'Xingyi', 'Aksu', 'Anshan', 'Anqing', 'An Shun', 'Alxa League', 'Baotou', 'Beihai', 'Bangda', 'Baoshan', 'Guangzhou', 'Changde', 'Zhengzhou', 'Changchun', 'Chaoyang', 'Chifeng', 'Changzhi', 'Chongqing', 'Changsha', 'Chengdu', 'Changzhou', 'Datong', 'Daxian', 'Daocheng County', 'Dandong', 'Chengde', 'Diqing', 'Dalian', 'Dali City', 'Dunhuang', 'Dongying', 'Dongsheng', 'Dayong', 'Enshi', "Yan'an", 'Fuzhou', 'Fuyang', 'Fuoshan', 'Fuyuan', 'Fuyun', 'Golmud', 'Guang Yuan', 'Haikou', 'Heihe', 'Hohhot', 'Hefei', 'Hangzhou', 'Hailar', 'Ul

In [13]:
print(select_best_cities_from_image(tokyo, list(city_dict.keys())[:20], 3))

['skyscraper', 'city', 'skyline', 'cityscape', 'tallest', 'architecture', 'tower', 'downtown', 'office', 'travel']
[('Kunduz', 0.0020057306590257878), ('Nakhichevan', 0.0019505851755526658), ('Baku', 0.0019181904239948184), ('Qabala', 0.0018791105543376136), ('Gyandzha', 0.0018023121089626407), ('Kabul', 0.0017236603157991936), ('Lankaran', 0.0015746904301767947), ('Bamiyan', 0.001487994589110585), ('Yerevan', 0.0014004682815816538), ('Maimana', 0.0013871374527112232), ('Kandahar', 0.0013395190799790904), ('Mazar-I-Sharif', 0.0013057929724596391), ('Faizabad', 0.0012985645597164215), ('Zaranj', 0.001201923076923077), ('Sharana', 0.0012012012012012011), ('Herat', 0.0011586366708506324), ('Gyoumri', 0.0007451564828614009), ('Chakcharan', 0.0007369196757553427), ('Bost', 0.0), ('Farah', 0.0)]


In [14]:
capitals = requests.get('https://en.wikipedia.org/wiki/List_of_national_capitals_in_alphabetical_order')

In [15]:
soup = BeautifulSoup(capitals.content, "lxml")

In [16]:
table_cap = soup.find('table', {"class" : "wikitable sortable"})


capitals = []
for row in table_cap.find_all('tr'):
    cols = row.find_all('td')

    if len(cols) == 3:
        capital = cols[0].text.strip()
        #capital = regex.findall(r'[\W]+', capital)[0]
        #capital = regex.findall(r'?!a', capital)
        capitals.append((capital, cols[1].text.strip()))


print(capitals)

[('Abu Dhabi', 'United Arab Emirates'), ('Abuja', 'Nigeria'), ('Accra', 'Ghana'), ('Adamstown', 'Pitcairn'), ('Addis Ababa', 'Ethiopia'), ('Algiers', 'Algeria'), ('Alofi', 'Niue'), ('Amman', 'Jordan'), ('Amsterdam', 'Netherlands'), ('Andorra la Vella', 'Andorra'), ('Ankara', 'Turkey'), ('Antananarivo', 'Madagascar'), ('Apia', 'Samoa'), ('Ashgabat', 'Turkmenistan'), ('Asmara', 'Eritrea'), ('Astana', 'Kazakhstan'), ('Asunción', 'Paraguay'), ('Athens', 'Greece'), ('Avarua', 'Cook Islands'), ('Baghdad', 'Iraq'), ('Baku', 'Azerbaijan'), ('Bamako', 'Mali'), ('Bandar Seri Begawan', 'Brunei'), ('Bangkok', 'Thailand'), ('Bangui', 'Central African Republic'), ('Banjul', 'Gambia'), ('Basse-Terre', 'Guadeloupe'), ('Basseterre', 'Saint Kitts and Nevis'), ('Beijing', 'China'), ('Beirut', 'Lebanon'), ('Belgrade', 'Serbia'), ('Belmopan', 'Belize'), ('Berlin [1]', 'Germany'), ('Bern', 'Switzerland'), ('Bishkek', 'Kyrgyzstan'), ('Bissau', 'Guinea-Bissau'), ('Bogotá', 'Colombia'), ('Brasília', 'Brazil'),

In [17]:
def save_word_list(cities):
    ret = {}
    for city in cities:
        startURL = 'https://en.wikipedia.org/wiki/'
        result = requests.get(startURL + city)
        if result.status_code == 200:
        

            c = result.content

            soup = BeautifulSoup(c, "lxml")
            text = soup.find(id='bodyContent').text
            ret[city] = text.split()
            print(city)
    return ret
    

In [18]:
capitals_name = map(lambda x: x[0], capitals)

pickle.dump(save_word_list(capitals_name), open("test.bin", "wb+"))

Abu Dhabi
Abuja
Accra
Adamstown
Addis Ababa
Algiers
Alofi
Amman
Amsterdam
Andorra la Vella
Ankara
Antananarivo
Apia
Ashgabat
Asmara
Astana
Asunción
Athens
Avarua
Baghdad
Baku
Bamako
Bandar Seri Begawan
Bangkok
Bangui
Banjul
Basse-Terre
Basseterre
Beijing
Beirut
Belgrade
Belmopan
Bern
Bishkek
Bissau
Bogotá
Brasília
Bratislava
Brazzaville
Bridgetown
Brussels
Bucharest
Budapest
Buenos Aires
Bujumbura
Cairo
Canberra
Caracas
Castries
Cayenne
Charlotte Amalie
Chișinău
Cockburn Town
Conakry
Copenhagen
Dakar
Damascus
Dhaka
Dili
Djibouti
Doha
Douglas
Dublin
Dushanbe
Edinburgh of the Seven Seas
Episkopi Cantonment
Flying Fish Cove
Fort-de-France
Freetown
Funafuti
Gaborone
George Town
Georgetown
Georgetown
Gibraltar
Grozny
Guatemala City
Gustavia
Hagåtña
Hamilton
Hanga Roa
Hanoi
Harare
Hargeisa
Havana
Helsinki
Hong Kong
Honiara
Islamabad
Jakarta
Jamestown
Juba
Kabul
Kampala
Kathmandu
Khartoum
Kiev
Kigali
King Edward Point
Kingston
Kingston
Kingstown
Kinshasa
Kuwait City
Libreville
Lilongwe
Lima
L

In [20]:
def get_most_likely_cities_file(image, file, num):
    ret = []
    words_list = pickle.load(open(file, 'rb'))
    words = find_best_words(image)
    for city in words_list:
        total = 0

        count = Counter(words_list[city])
        for word in words:
            total+=count[word]
        score = float(total) / float(len(words_list[city]))
        ret.append((city, score))
    return sorted(ret, key=lambda tup: tup[1], reverse=True)[:num]
        

        
beach = 'https://media-cdn.tripadvisor.com/media/photo-s/0e/85/48/e6/seven-mile-beach-grand.jpg'
counts = get_most_likely_cities_file(beach, 'capitals_wikipedia.bin', 10)

print(counts)
    

[('Gustavia', 0.009615384615384616), ('Marigot', 0.008333333333333333), ('Palikir', 0.007194244604316547), ('Malé', 0.005909389363099146), ('Tarawa', 0.0050533408197641775), ('Moroni', 0.004975124378109453), ('Edinburgh of the Seven Seas', 0.0043859649122807015), ('Basseterre', 0.0038576939562794685), ('Malabo', 0.0036665851869958446), ('Alofi', 0.003629764065335753)]
